In [1]:
from pathlib import Path
from ultralytics import YOLO
import cv2
import time

## Load Model

In [2]:
model_path = Path("../models/best_model.pt")

model = YOLO(str(model_path))
print("Model loaded:", model_path)

Model loaded: ../models/best_model.pt


## Real-time Inference

In [ ]:
# Webcam settings
CAMERA_INDEX = 0
IMG_SIZE = 640
CONF_THRESH = 0.35
IOU_THRESH = 0.5

cap = cv2.VideoCapture(CAMERA_INDEX)

if not cap.isOpened():
    raise RuntimeError("❌ Could not open webcam")

print("Real-time person detection started (press 'q' to quit)")

prev_time = time.time()

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("⚠️ Failed to grab frame")
            break

        # Run inference (single-class person model)
        result = model.predict(
            source=frame,
            imgsz=IMG_SIZE,
            conf=CONF_THRESH,
            iou=IOU_THRESH,
            verbose=False
        )[0]

        # Annotated output (boxes + scores)
        annotated_frame = result.plot()

        # Person count
        person_count = len(result.boxes) if result.boxes is not None else 0

        # FPS calculation
        curr_time = time.time()
        fps = 1 / (curr_time - prev_time)
        prev_time = curr_time

        # Overlay info
        cv2.putText(
            annotated_frame,
            f"Persons: {person_count}",
            (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 255, 0),
            2
        )

        cv2.putText(
            annotated_frame,
            f"FPS: {fps:.1f}",
            (10, 65),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.9,
            (255, 0, 0),
            2
        )

        # Display
        cv2.imshow("Real-Time Person Detection", annotated_frame)

        # Exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    cap.release()
    cv2.destroyAllWindows()

Real-time person detection started (press 'q' to quit)
Camera released and resources cleaned up


: 